In [1]:
import findspark
findspark.init('/opt/spark') 

In [2]:
import pandas as pd
findspark.find()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('cluster').getOrCreate()

23/05/16 07:33:55 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.153.129 instead (on interface ens33)
23/05/16 07:33:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/05/16 07:33:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
pip install cassandra-driver

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install uuid

Note: you may need to restart the kernel to use updated packages.


In [5]:
!pip install pyspark

In [6]:
import uuid
df = spark.read.csv("MR.csv", header=True) 

In [7]:
df.show()

+-------+--------+----------+----+-----+---+----------+
|    Lat|     Lon|BaseString|year|month|day|prediction|
+-------+--------+----------+----+-----+---+----------+
|40.0653|-74.0462|    B02512|2014|    8| 23|         0|
| 40.067|-74.0687|    B02512|2014|    8| 31|         3|
|40.0875|-74.0658|    B02512|2014|    8| 31|         3|
|40.0893| -74.043|    B02512|2014|    8| 31|         3|
|40.0894|-74.0422|    B02512|2014|    8| 30|         3|
|40.0938| -74.037|    B02512|2014|    8| 10|         4|
| 40.094|-74.0555|    B02512|2014|    8|  1|         1|
|40.0944|-74.0453|    B02512|2014|    8| 30|         3|
|40.1077|-74.0525|    B02598|2014|    8| 22|         0|
|40.1109|-74.0389|    B02598|2014|    8|  9|         4|
|40.1117|-74.0342|    B02512|2014|    8| 22|         0|
|40.1119|-74.0347|    B02512|2014|    8| 24|         3|
| 40.112|-74.0343|    B02512|2014|    8|  3|         1|
|40.1187|-74.0435|    B02598|2014|    8| 16|         2|
|40.1301|-74.0433|    B02598|2014|    8|  2|    

In [8]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from cassandra.query import BatchStatement

In [21]:

# Connect to the Cassandra cluster
cluster = Cluster(['localhost'])
session = cluster.connect()



In [10]:
# Create a keyspace
keyspace = 'mykeyspace'  # Replace with your desired keyspace name

query_create_keyspace = f"CREATE KEYSPACE IF NOT EXISTS {keyspace} WITH REPLICATION = {{ 'class' : 'SimpleStrategy', 'replication_factor' : 1 }}"
session.execute(query_create_keyspace)

query_use_keyspace = f"USE {keyspace}"
session.execute(query_use_keyspace)

In [45]:
# Create a table
table_name = 'mytable'  # Replace with your desired table name

# Create the table
query_create_table = f"CREATE TABLE IF NOT EXISTS {keyspace}.{table_name} (id UUID, Lat double, Lon double, BaseString TEXT, year INT, month INT, day INT, prediction INT, PRIMARY KEY((prediction),id))"
session.execute(query_create_table)

In [46]:
rows = df.collect()

In [48]:
# Insert rows into the table
for row in rows:
    id_value = uuid.uuid4()
    query_insert = f"INSERT INTO {keyspace}.{table_name} (id, Lat, Lon, BaseString, year, month, day, prediction) VALUES ({id_value}, {row.Lat}, {row.Lon}, '{row.BaseString}', {row.year}, {row.month}, {row.day}, {row.prediction})"
    session.execute(query_insert)

In [38]:
from datetime import datetime, timedelta
from cassandra.cluster import Cluster
from collections import defaultdict


# Define the date range for the query
start_date = datetime(2014, 8, 17)
end_date = datetime(2014, 8, 27)

# Prepare the query
query = f"SELECT Lat, Lon FROM mykeyspace.mytable WHERE year = {start_date.year} AND month = {start_date.month} AND day >= {start_date.day} AND day <= {end_date.day} ALLOW FILTERING"

# Execute the query
result = session.execute(query)

# Perform grouping and counting in Python
traffic_count = defaultdict(int)
for row in result:
    lat = row.lat
    lon = row.lon
    traffic_count[(lat, lon)] += 1

# Print the traffic count
for (lat, lon), count in traffic_count.items():
    print(f"Lat: {lat}, Lon: {lon}, Count: {count}")



Lat: 40.7252, Lon: -73.978, Count: 15
Lat: 40.7449, Lon: -74.005, Count: 6
Lat: 40.7509, Lon: -73.9706, Count: 3
Lat: 40.7267, Lon: -73.9505, Count: 3
Lat: 40.6988, Lon: -73.9346, Count: 3
Lat: 40.7071, Lon: -74.0046, Count: 6
Lat: 40.696, Lon: -73.9075, Count: 3
Lat: 40.703, Lon: -73.9946, Count: 12
Lat: 40.8125, Lon: -73.9295, Count: 3
Lat: 40.7542, Lon: -73.9648, Count: 9
Lat: 40.7527, Lon: -73.9859, Count: 6
Lat: 40.7469, Lon: -73.9806, Count: 3
Lat: 40.6461, Lon: -73.7805, Count: 3
Lat: 40.6892, Lon: -73.9371, Count: 3
Lat: 40.7492, Lon: -73.9941, Count: 3
Lat: 40.7582, Lon: -73.9696, Count: 3
Lat: 40.7707, Lon: -73.9615, Count: 3
Lat: 40.7433, Lon: -73.9742, Count: 3
Lat: 40.7201, Lon: -73.9885, Count: 3
Lat: 40.7024, Lon: -73.9666, Count: 3
Lat: 40.7683, Lon: -73.9838, Count: 3
Lat: 40.7631, Lon: -73.9742, Count: 3
Lat: 40.7036, Lon: -73.9315, Count: 3
Lat: 40.7557, Lon: -73.9934, Count: 3
Lat: 40.7086, Lon: -73.9559, Count: 3
Lat: 40.6917, Lon: -73.9844, Count: 6
Lat: 40.7299, 

In [41]:
# Define the date range for the query
start_date = datetime(2014, 8, 21)
end_date = datetime(2014, 8, 31)

# Prepare the query
query = f"SELECT Lat, Lon FROM mykeyspace.mytable WHERE year = {start_date.year} AND month = {start_date.month} AND day >= {start_date.day} AND day <= {end_date.day} ALLOW FILTERING"

# Execute the query
result = session.execute(query)

# Perform grouping and counting in Python
traffic_count = defaultdict(int)
for row in result:
    lat = row.lat
    lon = row.lon
    traffic_count[(lat, lon)] += 1

# Find the busiest point(s)
max_count = max(traffic_count.values())
busiest_points = [(lat, lon) for (lat, lon), count in traffic_count.items() if count == max_count]

# Print the busiest point(s)
for point in busiest_points:
    lat, lon = point
    print(f"Lat: {lat}, Lon: {lon}, Count: {max_count}")


Lat: 40.6448, Lon: -73.7819, Count: 156
